In [ ]:
from pathlib import Path
import sys, csv
from datetime import datetime
#src_path = str(Path.cwd().parent / "src")
#sys.path.append(src_path)

import numpy as np
import pandas as pd
import random
#from scipy.stats import poisson

from sklearn.pipeline import Pipeline
#!pip install scikit-survival

import plotly.express as px

from Supply_chain_disruption_model import SimulationModel
import Supply_chain_disruption_model as scd
import Centrality as cen

# Simulation

Generate data.

In [ ]:
# dim: number of firms
dim = 100
# nb_s: number of sectors
nb_s = 5

random.seed(42)
adj, A, C, sector = scd.generate_data(dim, nb_s)

Define model parameters.

In [ ]:
# p: The proportion (percentage/100) of firms that are damaged by the disruption. (default is 0.1)
# damage_level: The average amount of damage inflicted on affected firms. On average, (100*damage_level)% of the 
#     production capacity is damaged. (default is 0.2)
# margin: Specifies the margin around the given average damage level. The actual damage will lie between 
#     (damage_level - margin) and (damage_level + margin), truncated from below by zero and from above by one. 
#     (default is 0.1)
# tau: The number of days over which the inventory is restored to the target value. (default is 6)
# k: The average target inventory of a firm, specified as number of days of product use. (default is 9)
#     value=9 from paper inoue_firm-level_2019
# gamma: The recovery rate of damaged firms. (default is 0.5)
# sigma: The number of days without recovery and production in the firms damaged after the disruption. (default is 6)
# alpha: The number of days a firm tolerates a negative inventory of a supplier, before it tries to replace the supplier. 
#     (default is 2)
# u: Each firm has on average (100*u)% capacity utilization. This is used to assign a maximum possible production 
#     capacity to each firm. (default is 0.8)
# max_init_inventory: Whether firms initially have a full inventory or not. (default is True)
# fixed_target_inventory: Whether the target inventory value is fixed or determined on the previous day's realized 
#     demand. (default is True)
# nb_iter: The number of iterations (days) to run the simulation. (default is 100)
param = {"p": 0.1, "damage_level": 0.2, "margin": 0.1, "tau": 6, "k": 9, "gamma": 0.5, "sigma": 6, "alpha": 2, 
         "u": 0.8, "max_init_inventory": False, "fixed_target_inventory": True, "nb_iter": 1*365}

Execute the simulation.

In [ ]:
# intialize the model
mdl = SimulationModel(A, sector, C, **param)

# time the simulation
start=datetime.now()  

# run the model
mdl.run_simulation(print_iter=False)

print(f"runtime: {datetime.now()-start}")

In [ ]:
print(f"{len(mdl.damaged_ind)} firms were damaged by the disruption, which is {100*len(mdl.damaged_ind)/dim :.1f}%.")
print(f"{len(mdl.defaults)} out of {dim} firms defaulted, which is {100*len(mdl.defaults)/dim :.1f}%.")

damaged_and_defaulted = list(set(mdl.damaged_ind) & set(mdl.defaults.keys()))

if len(mdl.damaged_ind) > 0:
    perc_damaged_and_defaulted_of_damaged = 100*len(damaged_and_defaulted)/len(mdl.damaged_ind)
    print(f"Of the damaged firms, {perc_damaged_and_defaulted_of_damaged :.1f}% defaulted, "
          f"{100 - perc_damaged_and_defaulted_of_damaged :.1f}% survived.")

if len(mdl.defaults)> 0:
    perc_damaged_and_defaulted_of_defaulted = 100*len(damaged_and_defaulted)/len(mdl.defaults)
    print(f"Of the defaulted firms, {perc_damaged_and_defaulted_of_defaulted :.1f}% had been damaged, "
          f"{100 - perc_damaged_and_defaulted_of_defaulted :.1f}% had not been damaged. \n")

    print(f"This means {100 - perc_damaged_and_defaulted_of_defaulted :.1f}% of the firms that defaulted did so due to "

          f"network propagation of the damage.")

#print(f"damaged firms that also defaulted: \n{sorted(damaged_and_defaulted)}\n")
#print(f"damaged (ind): \n{mdl.damaged_ind} \n")
#print(f"defaults (ind: iteration): \n{mdl.defaults} \n")
#print(f"defaulted firms: \n{sorted(mdl.defaults.keys())} \n")
#print(f"default times: \n{sorted(mdl.defaults.values())}")

In [ ]:
print(f"defaulted firms: {sorted(list(mdl.defaults.keys()))}\n")
print(f"damaged firms: {sorted(mdl.damaged_ind)}")

In [ ]:
# filter which firms are plotted
# select_firms = "all" or [], "damaged", "not_damaged", arr of specific firm indices
select_firms = "all"
mdl.plot_capacity(relative=True, col_by_sector=False, show_leg=True, select_firms=select_firms)

In [ ]:
#mdl.plot_capacity(relative=True, col_by_sector=True, show_leg=True, select_firms=[])

In [ ]:
#mdl.plot_capacity(relative=False, col_by_sector=False)

# Calculate average production loss

In [ ]:
# relative production capacity data
prodcap_df = mdl.get_prod_capacity_df(relative=True, select_firms=[])
prodcap_df

In [ ]:
prodcap_df["prod_loss"] = 1 - prodcap_df["prod_cap"]
df_filtered = prodcap_df[prodcap_df["prod_loss"] >= 0].copy()
df_filtered

In [ ]:
df = df_filtered.groupby("firm")["prod_loss"].sum().to_frame()
df.reset_index(inplace=True)
df.rename(columns={"prod_loss": "tot_prod_loss"}, inplace=True)
df["avg_prod_loss"] = df["tot_prod_loss"]/(mdl.param["nb_iter"] + 1)
df = df.merge(pd.DataFrame(np.transpose([mdl.sector, np.array(range(mdl.dim()))]), columns=["sector","firm"]), on='firm')
df = df[['firm', 'sector', 'avg_prod_loss']]
df

In [ ]:
damaged = np.zeros(dim).astype(int)
damaged[mdl.damaged_ind] = 1
defaulted = np.zeros(dim).astype(int)
defaulted[list(mdl.defaults.keys())] = 1
df["damaged"] = damaged
df["defaulted"] = defaulted

df

## PN score

In [ ]:
scores = cen.PN_score(adj).reshape(1,-1)[0]

In [ ]:
fig = px.scatter(x=scores, y=df["avg_prod_loss"].values, color=damaged, symbol=defaulted, 
                 title="Absolute values in adjacency matrix",
                 labels={'x':'PN score', 'y':'mean daily production loss', 'color':'damaged', 'symbol':'defaulted'}) 
fig.show()

In [ ]:
adj_bin = np.copy(adj)
adj_bin[adj_bin > 0] = 1
adj_bin[adj_bin < 0] = -1

scores_bin = cen.PN_score(adj_bin).reshape(1,-1)[0]

In [ ]:
fig = px.scatter(x=scores_bin, y=prod_loss["avg_prod_loss"].values, color=damaged, symbol=defaulted, 
                 title="Absolute values in adjacency matrix",
                 labels={'x':'PN score', 'y':'mean daily production loss', 'color':'damaged', 'symbol':'defaulted'}) 
fig.show()

# Multiple simulations

In [ ]:
random.seed(42)

In [ ]:
# dim: number of firms
dim = 100
# nb_s: number of sectors
nb_s = 8

adj, A, C, sector = scd.generate_data(dim, nb_s)

In [ ]:
# p = [0.1, 0.2, 0.3]
# damage_level = [0.2, 0.4, 0.8]
# k = [3, 9]
# alpha = 3  # [3, 5]
# u = [0.7, 0.8]
# max_init_inventory = False
# nb_iter = 3*365
# 
# nb_exp = 36
# 
# setup = pd.DataFrame(columns=['p', 'damage_level', 'k', 'alpha', 'u', 'max_init_inventory', 'nb_iter'])
# setup["p"] = np.repeat(p, nb_exp/3)
# setup["damage_level"] = np.repeat([np.repeat(damage_level, nb_exp/3/3)], 3, axis=0).reshape(-1,1)
# setup["k"] = np.repeat([np.repeat(k, nb_exp/3/3/2)], 3*3, axis=0).reshape(-1,1)
# #setup["alpha"] = np.repeat([np.repeat(alpha, nb_exp/3/3/2/2)], 3*3*2, axis=0).reshape(-1,1)
# #setup["u"] = np.repeat([np.repeat(u, nb_exp/3/3/2/2/2)], 3*3*2*2, axis=0).reshape(-1,1)
# setup["alpha"] = alpha
# setup["u"] = np.repeat([np.repeat(u, nb_exp/3/3/2/2)], 3*3*2, axis=0).reshape(-1,1)
# setup["max_init_inventory"] = max_init_inventory
# setup["nb_iter"] = nb_iter
# setup["setup_nb"] = range(setup.shape[0])
# setup.head(10)
# setup.to_csv('sim_setup.csv', index=False)

In [ ]:
sim_setup_og = pd.read_csv('sim_setup.csv')
#sim_setup.to_dict(orient='list')
setup_nbs = sim_setup_og["setup_nb"]
sim_setup = sim_setup_og[['p', 'damage_level', 'k', 'alpha', 'u', 'max_init_inventory', 'nb_iter']]
sim_setup

In [ ]:
nb_rep = 5

for i in range(sim_setup.shape[0]):
    
    for j in range(nb_rep):
    
        param = dict(sim_setup.loc[i])
        mdl = SimulationModel(A, sector, C, **param)
        mdl.run_simulation(print_iter=False)
        
        prodcap_df = mdl.get_prod_capacity_df(relative=True)
        
        prodcap_df["prod_loss"] = 1 - prodcap_df["prod_cap"]
        df_filtered = prodcap_df[prodcap_df["prod_loss"] >= 0].copy()
        
        df = df_filtered.groupby("firm")["prod_loss"].sum().to_frame()
        df.reset_index(inplace=True)
        df["avg_prod_loss"] = df["prod_loss"]/(mdl.param["nb_iter"] + 1)
        df = df.merge(pd.DataFrame(np.transpose([mdl.sector, np.array(range(mdl.dim()))]), columns=["sector","firm"]), 
                      on='firm')
        df = df[['firm', 'sector', 'avg_prod_loss']]
        
        damaged = np.zeros(mdl.dim()).astype(int)
        damaged[mdl.damaged_ind] = 1
        defaulted = np.zeros(mdl.dim()).astype(int)
        defaulted[list(mdl.defaults.keys())] = 1
        df["damaged"] = damaged
        df["defaulted"] = defaulted
        
        df["setup"] = setup_nbs.loc[i]
        df["rep"] = j
        
        if i == 0 and j == 0:
            df.to_csv('sim_results2.csv', index=False, header=True)
        else:
            df.to_csv('sim_results2.csv', mode='a', index=False, header=False)

In [ ]:
sim_res = pd.read_csv('sim_results.csv')
sim_res

Filter out the damaged firms.

In [ ]:
sim_res = sim_res[sim_res["damaged"] == 0]
sim_res

Take average over replications.

In [ ]:
df = sim_res.groupby(["firm","sector","setup"])["avg_prod_loss"].mean().to_frame()
df.reset_index(inplace=True)
df

In [ ]:
df = df.merge(sim_setup_og, left_on='setup', right_on='setup_nb')
df

In [ ]:
scores = cen.PN_score(adj).reshape(1,-1)[0]

adj_bin = np.copy(adj)
adj_bin[adj_bin > 0] = 1
adj_bin[adj_bin < 0] = -1
scores_bin = cen.PN_score(adj_bin).reshape(1,-1)[0]

In [ ]:
firm_scores = pd.DataFrame(np.transpose(np.array([range(mdl.dim()),scores])), columns=["firm","score"])
firm_scores["score_bin"] = scores_bin
df = df.merge(firm_scores, on='firm')
df

In [ ]:
fig = px.scatter(x=df.score, y=df.avg_prod_loss, title="Absolute values in adjacency matrix",
                 labels={'x':'PN score', 'y':'mean daily production loss'}) 
fig.show()

In [ ]:
fig = px.scatter(x=df.score_bin, y=df.avg_prod_loss, title="Binary values in adjacency matrix",
                 labels={'x':'PN score', 'y':'mean daily production loss'}) 
fig.show()

# Build network for visualization in presentation

In [ ]:
dim = 15
nb_s = 5
adj, A, C, sector = scd.generate_data(dim, nb_s)

In [ ]:
df = pd.DataFrame(columns=["supplier","customer"])

for i in range(dim):
    customer = i
    suppliers = A[i]
    for j in range(dim):
        if suppliers[j] > 0:
            df = pd.concat([df, pd.DataFrame([[int(j), i]], columns=["supplier","customer"])])

df

In [ ]:
#df.to_csv('links.csv', index=False)

In [ ]:
df_firms = pd.DataFrame(list(range(dim)), columns=["firm"])
df_firms["sector"] = sector
df_firms

In [ ]:
#df_firms.to_csv('firms.csv', index=False)